# Load Dependencies

In [2]:
import os
os.environ['WANDB_DISABLED']="true"

In [4]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

/root/miniconda3/envs/ugrip_env_v2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-08-18 06:28:20.031647: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-18 06:28:20.053331: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755498500.068598  101439 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755498500.073192  101439 cuda_blas.cc:1

### Load Dataset

In [5]:
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

### Load Model

In [6]:
model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [7]:
# Function to count trainable and total parameters
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0

    # Loop through all model parameters
    for _, param in model.named_parameters():
        all_model_params += param.numel()           # total params
        if param.requires_grad:                    # check if trainable
            trainable_model_params += param.numel()


    return f"trainable model parameters: {trainable_model_params}\n" \
           f"all model parameters: {all_model_params}\n" \
           f"percentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"


print(print_number_of_trainable_model_parameters(original_model))


trainable model parameters: 247577856
all model parameters: 247577856
percentage of trainable model parameters: 100.00%


## Test the Model with Zero Shot Inferencing

In [8]:
# Pick a test sample by index
index = 200

# Extract dialogue and human-written summary
dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

# Create summarization prompt
prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

# Tokenize the prompt into model inputs
inputs = tokenizer(prompt, return_tensors='pt')

# Generate summary using the model
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,     # limit output length
    )[0],
    skip_special_tokens=True   # clean special tokens
)

# Formatting line for readability
dash_line = '-'.join('' for x in range(100))

# Print input prompt, human summary, and model output
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')


---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation.

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.

Summary:

-------------------------------------------------------------------

# Full Fine Tuning

In [9]:
# Function to tokenize dialogue-summary pairs
def tokenize_function(example):
    start_prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '

    # Build input prompt for each dialogue
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]

    # Tokenize inputs (prompts) and labels (summaries)
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example

# Apply tokenization to all dataset splits (train/val/test)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Remove unused columns, keeping only tokenized data
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary'])

In [10]:
# Take first tokenized input (prompt + dialogue)
sample_input_id = tokenized_datasets['train']['input_ids'][0:1]

# Take first tokenized label (summary)
sample_label = tokenized_datasets['train']['labels'][0:1]

# Print dataset object details
print("tokenized_datasets: ", tokenized_datasets)

# Print sample input ID length and values
print("\nsample_input_id: ", len(sample_input_id[0]), sample_input_id)

# Print sample label length and values
print("\nsample_label: ", len(sample_label[0]), sample_label)

tokenized_datasets:  DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 500
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 1500
    })
})

sample_input_id:  512 [[12198, 1635, 1737, 8, 826, 3634, 5, 1713, 345, 13515, 536, 4663, 10, 2018, 6, 1363, 5, 3931, 5, 27, 31, 51, 7582, 12833, 77, 7, 5, 1615, 33, 25, 270, 469, 58, 1713, 345, 13515, 357, 4663, 10, 27, 435, 34, 133, 36, 3, 9, 207, 800, 12, 129, 3, 9, 691, 18, 413, 5, 1713, 345, 13515, 536, 4663, 10, 2163, 6, 168, 6, 25, 43, 29, 31, 17, 141, 80, 21, 305, 203, 5, 148, 225, 43, 80, 334, 215, 5, 1713, 345, 13515, 357, 4663, 10, 27, 214, 5, 27, 2320, 38, 307, 38, 132, 19, 1327, 1786, 6, 572, 281, 217, 8, 2472, 58, 1713, 345, 13515, 536, 4663, 10, 1548, 6, 8, 200, 194, 12, 1792, 2261, 21154, 19, 12, 253, 91, 81, 135, 778, 5, 264, 653, 12, 369, 44, 709, 72

In [11]:
# Print dataset shapes (rows, columns) for each split
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

# Print tokenized dataset object summary
print(tokenized_datasets)


Shapes of the datasets:
Training: (12460, 2)
Validation: (500, 2)
Test: (1500, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 500
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 1500
    })
})


## Fine-Tune the Model with the Preprocessed Dataset

In [12]:
# Create unique output directory using current timestamp
output_dir = f'./dialogue-summary-training-{str(int(time.time()))}'

# Define training hyperparameters and logging settings
training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    logging_steps=50,
    eval_strategy="epoch",   # <-- correct name in your version
    save_strategy="epoch",
)

# Initialize Trainer with model, args, and datasets
trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [13]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,29.805000,31.466999
2,29.800000,31.450001
3,29.280000,31.455000


TrainOutput(global_step=9345, training_loss=30.46565008025682, metrics={'train_runtime': 1956.6775, 'train_samples_per_second': 19.104, 'train_steps_per_second': 4.776, 'total_flos': 2.559622983450624e+16, 'train_loss': 30.46565008025682, 'epoch': 3.0})

In [14]:
# Save the fully fine-tuned model
trainer.model.save_pretrained(output_dir)

In [15]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load true original (pretrained) for baseline comparison
original_model = AutoModelForSeq2SeqLM.from_pretrained(
    "google/flan-t5-base", torch_dtype="bfloat16"
).to(device).eval()

# Load your fine-tuned model
instruct_model = AutoModelForSeq2SeqLM.from_pretrained(
    output_dir, torch_dtype="bfloat16", local_files_only=True
).to(device).eval()

In [16]:

# Pick test sample
index = 200
dialogue = dataset['test'][index]['dialogue']
human_baseline_summary = dataset['test'][index]['summary']

# Create summarization prompt
prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

# Tokenize input
# input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

# Generate summary with original model (zero-shot)
original_model_outputs = original_model.generate(
    input_ids=input_ids,
    generation_config=GenerationConfig(max_new_tokens=200, num_beams=1)
)
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

# Generate summary with fine-tuned instruct model
instruct_model_outputs = instruct_model.generate(
    input_ids=input_ids,
    generation_config=GenerationConfig(max_new_tokens=200, num_beams=1)
)
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

# Print human, original, and instruct model summaries
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')


`generation_config` default values have been modified to match model-specific defaults: {'pad_token_id': 0, 'eos_token_id': 1, 'decoder_start_token_id': 0}. If this is not desired, please set these values explicitly.


---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:
#Person1#: I'm thinking of upgrading my computer.
---------------------------------------------------------------------------------------------------
INSTRUCT MODEL:
You might want to upgrade your computer.


## Evaluate the Model Quantitatively (with ROUGE Metric)

In [17]:
# Load ROUGE metric for evaluation
rouge = evaluate.load('rouge')

# Select first 10 test dialogues and human summaries
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

# Initialize lists to store model outputs
original_model_summaries = []
instruct_model_summaries = []

# Generate summaries for each dialogue
for _, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

    # Original model summary
    original_model_outputs = original_model.generate(
        input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200)
    )
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    original_model_summaries.append(original_model_text_output)

    # Fine-tuned instruct model summary
    instruct_model_outputs = instruct_model.generate(
        input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200)
    )
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)
    instruct_model_summaries.append(instruct_model_text_output)

# Combine human and model summaries for comparison
zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries))

# Convert to pandas DataFrame
df = pd.DataFrame(zipped_summaries, columns=['human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries'])
df


,human_baseline_summaries,original_model_summaries,instruct_model_summaries
0,Ms. Dawson helps #Person1# to write a memo to ...,#Person1#: I need to take a dictation for you.,This memo is to be distributed to all employee...
1,In order to prevent employees from wasting tim...,#Person1#: I need to take a dictation for you.,This memo is to be distributed to all employee...
2,Ms. Dawson takes a dictation for #Person1# abo...,#Person1#: I need to take a dictation for you.,This memo is to be distributed to all employee...
3,#Person2# arrives late because of traffic jam....,The traffic jam at the Carrefour intersection ...,Taking public transport to work is a good idea.
4,#Person2# decides to follow #Person1#'s sugges...,The traffic jam at the Carrefour intersection ...,Taking public transport to work is a good idea.
5,#Person2# complains to #Person1# about the tra...,The traffic jam at the Carrefour intersection ...,Taking public transport to work is a good idea.
6,#Person1# tells Kate that Masha and Hero get d...,Masha and Hero are getting divorced.,Masha and Hero are getting divorced.
7,#Person1# tells Kate that Masha and Hero are g...,Masha and Hero are getting divorced.,Masha and Hero are getting divorced.
8,#Person1# and Kate talk about the divorce betw...,Masha and Hero are getting divorced.,Masha and Hero are getting divorced.
9,#Person1# and Brian are at the birthday party ...,"#Person1#: Happy birthday, Brian. #Person2#: I...",Brian's birthday is coming up.


In [18]:
# Compute ROUGE scores for the original model
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,  # aggregate scores across all examples
    use_stemmer=True      # apply stemming for better matching
)

# Compute ROUGE scores for the fine-tuned instruct model
instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True
)

# Print ROUGE evaluation results
print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)


ORIGINAL MODEL:
{'rouge1': 0.24089921652421653, 'rouge2': 0.11769053708439897, 'rougeL': 0.22001958689458687, 'rougeLsum': 0.22134175465057818}
INSTRUCT MODEL:
{'rouge1': 0.28647748040489973, 'rouge2': 0.13497482028216662, 'rougeL': 0.23619027925479535, 'rougeLsum': 0.23930701616185485}


# Parameter Efficient Fine-Tuning (PEFT)

In [19]:
from peft import LoraConfig, get_peft_model, TaskType

# Define the LoRA configuration
lora_config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,        #
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

In [20]:
# Apply LoRA to the original model
peft_model = get_peft_model(original_model, lora_config)

# Print number of trainable parameters
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 3538944
all model parameters: 251116800
percentage of trainable model parameters: 1.41%


## Train PEFT Adapter

In [21]:
import time
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Training arguments (Seq2Seq version)
peft_training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3,
    num_train_epochs=3,
    logging_steps=50,
    eval_strategy="epoch",   # <-- correct name in your version
    save_strategy="epoch",
)

# Initialize PEFT trainer
peft_trainer = Seq2SeqTrainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets.get("validation", None),
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [22]:
# Start LoRA fine-tuning
peft_trainer.train()

# Define path to save the trained model
peft_model_path = "./peft-dialogue-summary-checkpoint-local"

# Save model and tokenizer
peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

Epoch,Training Loss,Validation Loss
1,0.109200,0.091146
2,0.106500,0.084461
3,0.102800,0.083794


('./peft-dialogue-summary-checkpoint-local/tokenizer_config.json',
 './peft-dialogue-summary-checkpoint-local/special_tokens_map.json',
 './peft-dialogue-summary-checkpoint-local/spiece.model',
 './peft-dialogue-summary-checkpoint-local/added_tokens.json',
 './peft-dialogue-summary-checkpoint-local/tokenizer.json')

In [23]:
from peft import PeftModel, PeftConfig

# Load pre-trained base model and tokenizer
peft_model_base = AutoModelForSeq2SeqLM.from_pretrained(
    "google/flan-t5-base",
    torch_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

# Load LoRA-adapted PEFT model from checkpoint, frozen for inference
peft_model = PeftModel.from_pretrained(
    peft_model_base,
    './peft-dialogue-summary-checkpoint-local',
    torch_dtype=torch.bfloat16,
    is_trainable=False
)


In [24]:
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 0
all model parameters: 251116800
percentage of trainable model parameters: 0.00%


### Evaluate the Model Qualitatively (Human Evaluation)

In [25]:
# Move models to device
peft_model.to(device)

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 768)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 768)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_features=32, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=32, out_features=768, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
            

In [26]:
# Load true original (pretrained) for baseline comparison
original_model = AutoModelForSeq2SeqLM.from_pretrained(
    "google/flan-t5-base", torch_dtype="bfloat16"
).to(device).eval()

# Load your fine-tuned model
instruct_model = AutoModelForSeq2SeqLM.from_pretrained(
    output_dir, torch_dtype="bfloat16", local_files_only=True
).to(device).eval()

In [27]:
index = 200
dialogue = dataset['test'][index]['dialogue']
baseline_human_summary = dataset['test'][index]['summary']

# Prepare summarization prompt
prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

# Tokenize prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
# Generate summary using original model
original_model_outputs = original_model.generate(
    input_ids=input_ids,
    generation_config=GenerationConfig(max_new_tokens=200, num_beams=1)
)
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

# Generate summary using instruction-tuned model
instruct_model_outputs = instruct_model.generate(
    input_ids=input_ids,
    generation_config=GenerationConfig(max_new_tokens=200, num_beams=1)
)
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

# Generate summary using PEFT (LoRA) model
peft_model_outputs = peft_model.generate(
    input_ids=input_ids,
    generation_config=GenerationConfig(max_new_tokens=200, num_beams=1)
)
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

# Print comparison
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{baseline_human_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')
print(dash_line)
print(f'PEFT MODEL:\n{peft_model_text_output}')


---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:
#Person1#: I'm thinking of upgrading my computer.
---------------------------------------------------------------------------------------------------
INSTRUCT MODEL:
You might want to upgrade your computer.
---------------------------------------------------------------------------------------------------
PEFT MODEL:
#Person1# suggests adding a painting program to #Person2#'s software and upgrading hardware. #Person2# also wants to upgrade the hardware. #Person1# suggests adding a CD-ROM drive too.


###  Evaluate the Model Quantitatively (with ROUGE Metric

In [28]:

# Select first 10 test dialogues and their human summaries
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

# Lists to store model outputs
original_model_summaries = []
instruct_model_summaries = []
peft_model_summaries = []

# Generate summaries for each dialogue
for idx, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)


    # Generate outputs for all models
    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

    peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

    # Append outputs to lists
    original_model_summaries.append(original_model_text_output)
    instruct_model_summaries.append(instruct_model_text_output)
    peft_model_summaries.append(peft_model_text_output)

# Combine human and model summaries into a DataFrame
zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries, peft_model_summaries))
df = pd.DataFrame(zipped_summaries, columns=['human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries', 'peft_model_summaries'])

df

,human_baseline_summaries,original_model_summaries,instruct_model_summaries,peft_model_summaries
0,Ms. Dawson helps #Person1# to write a memo to ...,#Person1#: I need to take a dictation for you.,This memo is to be distributed to all employee...,#Person1# asks Ms. Dawson to take a dictation ...
1,In order to prevent employees from wasting tim...,#Person1#: I need to take a dictation for you.,This memo is to be distributed to all employee...,#Person1# asks Ms. Dawson to take a dictation ...
2,Ms. Dawson takes a dictation for #Person1# abo...,#Person1#: I need to take a dictation for you.,This memo is to be distributed to all employee...,#Person1# asks Ms. Dawson to take a dictation ...
3,#Person2# arrives late because of traffic jam....,The traffic jam at the Carrefour intersection ...,Taking public transport to work is a good idea.,#Person2# got stuck in traffic again. #Person1...
4,#Person2# decides to follow #Person1#'s sugges...,The traffic jam at the Carrefour intersection ...,Taking public transport to work is a good idea.,#Person2# got stuck in traffic again. #Person1...
5,#Person2# complains to #Person1# about the tra...,The traffic jam at the Carrefour intersection ...,Taking public transport to work is a good idea.,#Person2# got stuck in traffic again. #Person1...
6,#Person1# tells Kate that Masha and Hero get d...,Masha and Hero are getting divorced.,Masha and Hero are getting divorced.,#Person1# tells Kate that Masha and Hero are g...
7,#Person1# tells Kate that Masha and Hero are g...,Masha and Hero are getting divorced.,Masha and Hero are getting divorced.,#Person1# tells Kate that Masha and Hero are g...
8,#Person1# and Kate talk about the divorce betw...,Masha and Hero are getting divorced.,Masha and Hero are getting divorced.,#Person1# tells Kate that Masha and Hero are g...
9,#Person1# and Brian are at the birthday party ...,"#Person1#: Happy birthday, Brian. #Person2#: I...",Brian's birthday is coming up.,Brian invites #Person1# to the party. Brian is...


In [29]:
# Load ROUGE metric
rouge = evaluate.load('rouge')

# Compute ROUGE for original model
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

# Compute ROUGE for instruction-tuned model
instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

# Compute ROUGE for PEFT model
peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

# Print results
print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)
print('PEFT MODEL:')
print(peft_model_results)


ORIGINAL MODEL:
{'rouge1': 0.24089921652421653, 'rouge2': 0.11769053708439897, 'rougeL': 0.22001958689458687, 'rougeLsum': 0.22134175465057818}
INSTRUCT MODEL:
{'rouge1': 0.28647748040489973, 'rouge2': 0.13497482028216662, 'rougeL': 0.23619027925479535, 'rougeLsum': 0.23930701616185485}
PEFT MODEL:
{'rouge1': 0.38339876176142457, 'rouge2': 0.13116253237675268, 'rougeL': 0.2959683345758466, 'rougeLsum': 0.29750462976498215}
